## Relax Challenge

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
#user data
data_user = pd.read_csv('takehome_users.csv',encoding = "ISO-8859-1")
data_user.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [5]:
#import user engagement data
data_engage = pd.read_csv('takehome_user_engagement.csv',encoding = "ISO-8859-1")
data_engage.time_stamp = pd.to_datetime(data_engage.time_stamp)
data_engage.index=data_engage.time_stamp
data_engage.drop(labels='time_stamp',axis=1,inplace=True)

In [6]:
data_engage.head()

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1


In [8]:
#Group by user, select week as a frequency, sum over week period
df_group = data_engage.groupby([pd.Grouper(freq='W'),'user_id']).sum()
df_group.head()

visited
time_stamp user_id         
2012-06-03 563            1
           1693           1
           1995           1
           2120           1
           2136           1

In [12]:
#select users with a sum of 3 or more as adopters
df_adopt = df_group[df_group.visited>=3].unstack(level=1).melt()
adopted_users = pd.DataFrame(df_adopt.user_id.unique(),index=range(df_adopt.user_id.unique().shape[0]),columns=['user_id'])
df_adopt.head()
adopted_users.head()

,user_id
0,1693
1,728
2,11764
3,5297
4,6171


In [17]:
#create df of features
df_join = data_user.merge(adopted_users,how='inner',left_on='object_id',right_on='user_id')
df_join.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id
0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2
1,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0,10
2,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN,20
3,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0,33
4,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN,42


In [18]:
#drop irrelevant columns
drop_cols = list(df_join.columns[0:4])
drop_cols.append('user_id')
df_join = df_join.drop(drop_cols,axis=1)
df_join.head()

,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,ORG_INVITE,1.396238e+09,0,0,1,316.0
1,ORG_INVITE,1.401833e+09,1,1,318,4143.0
2,SIGNUP,1.401364e+09,0,0,58,NaN
3,GUEST_INVITE,1.401518e+09,0,0,401,79.0
4,SIGNUP,1.401045e+09,1,0,235,NaN


In [20]:
#fill missing values in invited_by_user column
df_join['invited_by_user_id'].fillna(value=0,inplace=True)

In [21]:
#one hot encode creation_source feature
df_create = pd.get_dummies(df_join['creation_source'])
df_features = pd.concat([df_join,df_create],axis=1)
df_features.drop('creation_source',axis=1,inplace=True)

#convert columns to float64
for col in df_features.columns:
    df_features[col] = df_features[col].astype('float64')
df_features.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1.396238e+09,0.0,0.0,1.0,316.0,0.0,1.0,0.0,0.0,0.0
1,1.401833e+09,1.0,1.0,318.0,4143.0,0.0,1.0,0.0,0.0,0.0
2,1.401364e+09,0.0,0.0,58.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.401518e+09,0.0,0.0,401.0,79.0,1.0,0.0,0.0,0.0,0.0
4,1.401045e+09,1.0,0.0,235.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#scale data
scaler = StandardScaler()
features = scaler.fit_transform(df_features)

#fit PCA
pca = PCA()
components = pca.fit_transform(features)

In [23]:
np.sum(pca.explained_variance_ratio_[0:7])

0.9227803367525597

In [24]:
df_comp = pd.DataFrame(pca.components_,columns=df_features.columns,index=['PC-1','PC-2','PC-3','PC-4','PC-5','PC-6','PC-7','PC-8','PC-9','PC-10'])
#absolute values of correlation with principal components
best_features = np.absolute(df_comp[np.absolute(df_comp) > 0.1])
best_features.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
PC-1,NaN,NaN,NaN,NaN,0.637653,0.232628,0.486575,0.233976,0.390324,0.296733
PC-2,NaN,0.695712,0.69891,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PC-3,0.116892,NaN,NaN,NaN,NaN,0.784060,0.590037,NaN,NaN,NaN
PC-4,NaN,NaN,NaN,0.261476,NaN,NaN,NaN,0.108246,0.648711,0.697782
PC-5,0.230407,NaN,NaN,0.219055,NaN,NaN,NaN,0.825688,0.296054,0.350811


In [25]:
#sum correlations to get a relative estimate of the feature importance
best_features.head(7).sum(axis=0).sort_values(ascending=False)

org_id                        1.792770
SIGNUP_GOOGLE_AUTH            1.705164
last_session_creation_time    1.700941
SIGNUP                        1.524752
PERSONAL_PROJECTS             1.438816
ORG_INVITE                    1.076612
GUEST_INVITE                  1.016688
enabled_for_marketing_drip    0.698910
opted_in_to_mailing_list      0.695712
invited_by_user_id            0.637653
dtype: float64

Features Organization id, signup, last session creation time, personal projects, org_invite, guest_invite, are the most predictive of future user adoption.